# Average Episode BPM
> Tracking average episode BPM over time.

- toc: true 
- badges: true
- comments: false
- categories: [asot, bpm]
- image: images/chart-preview.png

In [11]:
#hide
import os
import yaml
import spotipy
import json
import altair as alt
import numpy as np
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

with open('spotipy_credentials.yaml', 'r') as spotipy_credentials_file:
    credentials = yaml.safe_load(spotipy_credentials_file)
    os.environ["SPOTIPY_CLIENT_ID"] = credentials['spotipy_credentials']['spotipy_client_id']
    os.environ["SPOTIPY_CLIENT_SECRET"] = credentials['spotipy_credentials']['spotipi_client_seret']

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

asot_radio_id = '25mFVpuABa9GkGcj9eOPce'

albums = []
results = sp.artist_albums(asot_radio_id, album_type='album')
albums.extend(results['items'])
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])
seen = set()  # to avoid dups
for album in albums:
    name = album['name']
    if name not in seen:
        seen.add(name)

singles = []
results = sp.artist_albums(asot_radio_id, album_type='single')
singles.extend(results['items'])
while results['next']:
    results = sp.next(results)
    singles.extend(results['items'])
seen = set()  # to avoid dups
for single in singles:
    name = single['name']
    if name not in seen:
        seen.add(name)

episodes = singles + albums

episodes.sort(key=lambda x: x['release_date']) # Sort by release date

## Introduction

Previously on "_How has A State of Trance changed over time?_" we poked the Spotify Web API and returned with every episode of A State of Trance. In this post, we'll examine how the show's BPM has changed over time.

## On Averages

First, some housekeeping items.

In the "Methodology" post we learned Spotify's catalogue of A State of Trance, while it seems to contain every episode, does not have every track in some episodes.

Due to the nature of averages, this will cause our calculations to be skewed somewhat. Filling in the blanks is left as an exercise for the reader.

## Calculating

Let's find the average BPM of each episode and graph it over time.

We'll ask Spotify for the BPM of every track in an episode, add it all up, and divide by the number of tracks - removing the voiceovers and interludes of course! For the first 10 episodes, this looks like:

In [12]:
episode_avg_bpm = []

for episode in episodes[:10]:
    try:
        episode_bpm = 0
        tracks_counted = 0
        for track in sp.album_tracks(episode['uri'])['items']:
            if "A State Of Trance" in track['name']:
                continue
            else:
                episode_bpm += sp.audio_features(track['uri'])[0]['tempo']
                tracks_counted += 1
        avg = episode_bpm/tracks_counted
        print(episode['name'], avg)
        episode_avg_bpm.append(avg)
    except:
        pass

A State Of Trance Episode 000 135.77875
A State Of Trance Episode 001 138.86957142857142
A State Of Trance Episode 002 135.95900000000003
A State Of Trance Episode 003 135.295
A State Of Trance Episode 004 137.3078
A State Of Trance Episode 005 137.122875
A State Of Trance Episode 006 136.12875
A State Of Trance Episode 007 137.459125
A State Of Trance Episode 008 136.409
A State Of Trance Episode 009 138.58033333333333


Rinse and repeat for 950+ episodes. Grab a tea or coffee, this could take a bit!

In [ ]:
episode_avg_bpm = []

for episode in episodes:
    try:
        episode_bpm = 0
        tracks_counted = 0
        for track in sp.album_tracks(episode['uri'])['items']:
            if "A State Of Trance" in track['name']:
                continue
            else:
                episode_bpm += sp.audio_features(track['uri'])[0]['tempo']
                tracks_counted += 1
        avg = episode_bpm/tracks_counted
        episode_avg_bpm.append(avg)
    except:
        pass

Instead of printing this, let's throw it into a quick Altair graph:

In [7]:
x = np.arange(len(episode_avg_bpm))   

source = pd.DataFrame({
  'episode': x,
  'avg bpm': np.array(episode_avg_bpm)
})

alt.Chart(source).mark_line().encode(
    alt.X('episode'),
    alt.Y('avg bpm', scale=alt.Scale(domain=(120, 150))),
).properties(
    title="A State of Trance - Average BPM of episode"
)

alt.Chart(...)

In [8]:
episode_bpm = 0
tracks_counted = 0
for track in sp.album_tracks('56qM5Y21wbvCW9l5GiAiaV')['items']:
    if "A State Of Trance" in track['name']:
        continue
    else:
        print(track['name'], ' - ', sp.audio_features(track['uri'])[0]['tempo'])
        episode_bpm += sp.audio_features(track['uri'])[0]['tempo']
        tracks_counted += 1
print(episode_bpm/tracks_counted)

Cloudwalking [ASOT 057] - Beatpusher Remix  -  141.61
Ligaya [ASOT 057] - Original Mix  -  141.784
Obsession [ASOT 057] - Original Mix  -  136.377
Gatex [ASOT 057] - Original Mix  -  184.848
Positron [ASOT 057] - Marco V Remix  -  139.264
148.7766


Finally, for fun, we can draw a line at 138 BPM to compare.

In [9]:
x = np.arange(len(episode_avg_bpm))

source = pd.DataFrame({
  'episode': x,
  'avg bpm': np.array(episode_avg_bpm)
})

source['138'] = 138

base = alt.Chart(source).mark_line().encode(
    alt.X('episode'),
    alt.Y('avg bpm', scale=alt.Scale(domain=(120, 150))),
).properties(
    title="A State of Trance - Average BPM of episode"
)

rule = alt.Chart(source).mark_rule(color='red').encode(
    y='138'
)

base + rule

alt.LayerChart(...)